# Kaggle Challange

In [ ]:
# importing Libraries

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout,BatchNormalization,UpSampling2D
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import PIL.Image as pim
from keras.applications import VGG19,ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam,SGD
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
# Loading DataSet
train_data=pd.read_csv('/content/train_data.csv')
train_label=pd.read_csv('/content/train_label.csv')
test_data=pd.read_csv('/content/test_data.csv',header=None)


In [ ]:
# Reshaping the data in the Desired format
label=train_label['label'][1:]
train_data=np.array(train_data)
train_data=train_data.reshape(9999,32,32,3)
testdata=np.array(test_data)
testdata=testdata.reshape(2000,32,32,3)
testdata.shape

(2000, 32, 32, 3)

In [ ]:
# Played with Model Configuration, changes the optimisers, validation split for diffrent results

# Model configuration
batch_size = 50
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 100
no_epochs = 100
optimizer = Adam()
# optimizer=SGD()
validation_split = 0.2
verbosity = 1

In [ ]:

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = train_data.astype('float32')
input_test=testdata.astype('float32')

# Normalize data
input_train = input_train / 255
input_test=input_test/255


In [ ]:
# Generated additional images using trained Data. Played with Multiple parameters in image generation

# image_generator = ImageDataGenerator(
# rotation_range=45,
# width_shift_range=.15,
# height_shift_range=.15,
# horizontal_flip=True,
# zoom_range=0.5)

image_generator = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")


In [ ]:
data_gen=image_generator.flow(input_train, label,
            batch_size=batch_size)

In [ ]:
# Callback for stopping the model when reaches desired efficiency. Played with multiple efficiencies for diffrent models. Higher effccienies have led the 
# model to overfit and resulting in bad score
ACCURACY_THRESHOLD = 0.85

class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('accuracy') > ACCURACY_THRESHOLD):
			self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()

In [ ]:
# Created model with few Convoluted layers.Although training accuracy was good ,score was bad. Model might be overfitting
validation_split = 0.3
# Create the model

model = Sequential()
model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

# Fit data to model
history = model.fit(input_train, label,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split, callbacks=[callbacks])


In [ ]:
# Trained the model on the more training data generated by the image generator. Played with multiple parameters. Score was better than the above model

# Create the model
validation_split = 0.2
no_epochs = 300
#third4.csv 29.85 with epoch 100
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='elu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='elu'))
model.add(Dense(128, activation='elu'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

# Fit data to model
history = model.fit(image_generator.flow(input_train, label,
            batch_size=batch_size),
            epochs=no_epochs,
            verbose=verbosity,
            # validation_split=validation_split, 
            callbacks=[callbacks])

In [ ]:
# Utilize Transfer Learning ResNet50 model and ran experiements for many variations in parameters.
# This would normally take 2 to 5 hours of training deoending on the parameters. Score has significantly improved using the transfer learning.

no_epochs=250
ACCURACY_THRESHOLD = 0.85
optimizer=Adam()
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(no_classes, activation='softmax'))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False
# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

# Fit data to model
history = model.fit(image_generator.flow(input_train, label,
            batch_size=batch_size),
            epochs=no_epochs,
            verbose=verbosity,
            # validation_split=validation_split, 
            callbacks=[callbacks])


Epoch 1/250
200/200 [==============================] - 91s 457ms/step - loss: 4.0565 - accuracy: 0.1052
Epoch 2/250
200/200 [==============================] - 93s 464ms/step - loss: 3.0679 - accuracy: 0.2309
Epoch 3/250
200/200 [==============================] - 93s 467ms/step - loss: 2.6285 - accuracy: 0.3171
Epoch 4/250
200/200 [==============================] - 94s 469ms/step - loss: 2.3199 - accuracy: 0.3817
Epoch 5/250
200/200 [==============================] - 94s 470ms/step - loss: 2.1359 - accuracy: 0.4127
Epoch 6/250
200/200 [==============================] - 94s 470ms/step - loss: 1.9773 - accuracy: 0.4535
Epoch 7/250
200/200 [==============================] - 94s 470ms/step - loss: 1.8581 - accuracy: 0.4854
Epoch 8/250
200/200 [==============================] - 94s 469ms/step - loss: 1.7678 - accuracy: 0.5036
Epoch 9/250
200/200 [==============================] - 94s 469ms/step - loss: 1.6911 - accuracy: 0.5224
Epoch 10/250
200/200 [==============================] - 94s 469m

In [ ]:
predictions=model.predict(input_test)
pred=tf.argmax(predictions, 1)
new=np.array(pred)

final=pd.concat([test_data.index.to_frame(),pd.DataFrame(new)],axis=1)
final.columns=['id','label']
FILE_NAME='final_11'+'.csv'
final.to_csv('/content/'+FILE_NAME,index=False)
